# 3-layer Neural Network for Classification 
without the deep learning framework (only python)

## 0.  Import dependency package

In [1]:
import numpy as np
import gzip
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline

## 1. Load data
The following functions are retrieved from https://stackoverflow.com/questions/40427435/extract-images-from-idx3-ubyte-file-or-gzip-via-python

In [2]:
def training_images():
    with gzip.open('data/train-images-idx3-ubyte.gz', 'r') as f:
        # first 4 bytes is a magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        # second 4 bytes is the number of images
        image_count = int.from_bytes(f.read(4), 'big')
        # third 4 bytes is the row count
        row_count = int.from_bytes(f.read(4), 'big')
        # fourth 4 bytes is the column count
        column_count = int.from_bytes(f.read(4), 'big')
        # rest is the image pixel data, each pixel is stored as an unsigned byte
        # pixel values are 0 to 255
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8)\
            .reshape((image_count, row_count, column_count))
        return images


def training_labels():
    with gzip.open('data/train-labels-idx1-ubyte.gz', 'r') as f:
        # first 4 bytes is a magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        # second 4 bytes is the number of labels
        label_count = int.from_bytes(f.read(4), 'big')
        # rest is the label data, each label is stored as unsigned byte
        # label values are 0 to 9
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

In [3]:
X_t = training_images()
Y_t = training_labels()

# normalization
X_t = X_t / 255

In [4]:
print(X_t.shape)

(60000, 28, 28)


## 2. Generate Neural Network

### 2.0 Activation Function

In [5]:
class ReLU:
    def prop(self, X):
        return np.maximum(0, X)
    
    def delta(self, X):
        return [1 for x in X if x >= 0 else 0]

class LeakyReLU:
    def __init__(self, hyper):
        self.hyper = hyper
        
    def prop(self, X):
        return np.maximum(self.hyper*X, X)
    
    def delta(self, X):
        return [1 for x in X if x >= 0 else self.hyper]

### 2.1 Layer_Dense

In [6]:
class Layer_dense:
    def __init__(self, input_size, output_size):
        self.W = np.random.normal(size=(input_size, output_size))
        self.b = np.random.normal(size=(1,output_size))
    
    def prop(self, X):
        return np.dot(X, self.W) + self.b
    
    def backprop(self,X):
        pass
    
    def update_W(self):
        pass

### 2.2 Softmax

In [7]:
class Softmax:
    def prop(self, X):
        return np.exp(X)/np.sum(np.exp(X))
    def backprop(self, Y):
        pass
    def delta(self, X):
        pass

### 2.3 Multi-Class Cross entropy function

In [8]:
class Cross_entropy:
    def prop(self, X, Y):
        return -1 * np.log(X[:,Y])
    
    def backprop(self, Y):
        pass
    
    def delta(self, X, Y):
        result = []
        for x,y in zip(X,Y):
            result.append([ -1 * (1/x) for i,z in enumerate(x) if i == y else 0])
        return np.ndarray(result)

### 2.4 Neural Network

In [ ]:
class NN:
    def __init__(self):
        self.layers = []
        loss_func = Cross_entropy()
        
    def add(self, layer):
        self.layers.append(layer)
    
    def prop(self):
        pass
    
    def backprop(self):
        pass
    
    def predict(self, X):
        for layer in layers:
            X = layer(X)
        

In [9]:
layer0 = Layer_dense(28*28,64)
layer1 = Layer_dense(64,10)

batch_x = x_t[0].reshape(1,-1)

output0 = layer0.prop(batch_x)
output1 = layer1.prop(output0)
output2 = softmax(output1)
output3 = cross_entropy(output2, y_t[0])

print(output2)
print(output3)

[[7.65314647e-42 5.98890143e-85 1.90993690e-84 3.66016458e-76
  1.00000000e+00 4.54972106e-80 5.38833821e-51 8.71449288e-23
  2.63601421e-40 2.93865567e-13]]
[182.69174151]
